# Section 73.1: Test Setup and Teardown within a unittest.TestCase

In [10]:
%cat unitest1.py

import unittest

class SomeTest(unittest.TestCase):
    def setUp(self):
        super(SomeTest, self).setUp()
        self.mock_data = [1, 2, 3, 4, 5]

    def test(self):
        self.assertEqual(len(self.mock_data), 5)

    def teraDown(self, sdfs):
        super(SomeTest, self).tearDown()
        self.mock_data = []

if __name__ == '__main__':
    unittest.main()

In [12]:
%run unitest1.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


# Section 73.2: Asserting on Exceptions

# Python 3 Testing: An Intro to unittest
Source: https://www.blog.pythonlibrary.org/2016/07/07/python-3-testing-an-intro-to-unittest/

In [14]:
%pycat mymath.py

TestCase 裡頭的每個 method 的開頭都要是 `test` 以及包含一個 assertion

In [36]:
!cat test_mymath.py

import mymath
import unittest

class TestAdd(unittest.TestCase):
    """
    Test the add function from the mymath library
    """

    def test_add_integers(self):
        result = mymath.add(1, 2)
        self.assertEqual(result, 3)

    def test_add_strings(self):
        result = mymath.add('abc', 'def')
        self.assertEqual(result, 'abcdef')

    def this_method_will_not_be_called(self):
        print('Catch me if you can.')

if __name__ == '__main__':
    unittest.main()


In [22]:
%run test_mymath.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


## Test with verbose output

In [23]:
%run test_mymath.py -v

test_add_integers (__main__.TestAdd) ... ok
test_add_strings (__main__.TestAdd) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


## 查看 unittest module 的參數

In [50]:
!python -m unittest -h | head -n 25

usage: python -m unittest [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                          [tests [tests ...]]

positional arguments:
  tests           a list of any number of test modules, classes and test
                  methods.

optional arguments:
  -h, --help      show this help message and exit
  -v, --verbose   Verbose output
  -q, --quiet     Quiet output
  --locals        Show local variables in tracebacks
  -f, --failfast  Stop on first fail or error
  -c, --catch     Catch Ctrl-C and display results so far
  -b, --buffer    Buffer stdout and stderr during tests

Examples:
  python -m unittest test_module               - run tests from test_module
  python -m unittest module.TestClass          - run tests from module.TestClass
  python -m unittest module.Class.test_method  - run specified test method
  python -m unittest path/to/test_file.py      - run tests from test_file.py

usage: python -m unittest discover [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                   

## 呼叫 unittest module 來跑 test module
- 沒有 `unittest.main()` 的 test script 也能直接跑
- 善用 `unittest.skip*` 來選擇要跑的 test function

In [37]:
!cat test_mymath_without_main.py

import sys
import mymath
import unittest

class TestAdd(unittest.TestCase):
    """
    Test the add function from the mymath library
    """

    def test_add_integers(self):
        result = mymath.add(1, 2)
        self.assertEqual(result, 3)


    def test_add_strings(self):
        result = mymath.add('abc', 'def')
        self.assertEqual(result, 'abcdef')


    def this_method_will_not_be_called(self):
        print('Catch me if you can.')


    @unittest.skip('Got skipped sadly')
    def test_func_1(self):
        print("I will be skipped")


    @unittest.skipUnless(sys.platform.startswith("win"), "Tested only on windows")
    def test_add_on_windows(self):
        result = mymath.add(100, 101)
        self.assertEqual(result, 201)

In [38]:
!python -m unittest test_mymath.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


In [40]:
!python -m unittest test_mymath_without_main.py -v

test_add_integers (test_mymath_without_main.TestAdd) ... ok
test_add_on_windows (test_mymath_without_main.TestAdd) ... skipped 'Tested only on windows'
test_add_strings (test_mymath_without_main.TestAdd) ... ok
test_func_1 (test_mymath_without_main.TestAdd) ... skipped 'Got skipped sadly'

----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK (skipped=2)


## 呼叫某 test module 裡特定的 function 

In [31]:
!python -m unittest test_mymath_without_main.TestAdd.test_add_strings

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


##  Test Discovery using Doctest


In [52]:
!cat my_docs.py | head -n 20

def add(a, b):
    """Return the addition of the arguments: a + b

    Examples:
        >>> add(-1, 10)
        9
        >>> add('a', 'b')
        'ab'
        >>> add(1, '2')
        Traceback (most recent call last):
          File "test.py", line 17, in <module>
            add(1, '2')
          File "test.py", line 14, in add
            return a + b
        TypeError: unsupported operand type(s) for +: 'int' and 'str'
    """
    return a + b


def subtract(a, b):


要測試 my_docs module 裡頭的 function, 要另外創一個 script 引進 doctest module 並把 my_docs module 當成 DocTestSuite

In [48]:
!cat test_doctests.py

import doctest
import my_docs
import unittest


def load_tests(loader, tests, ignore):
    tests.addTests(doctest.DocTestSuite(my_docs))
    return tests

In [53]:
!python -m unittest discover -v

add (my_docs)
Doctest: my_docs.add ... ok
subtract (my_docs)
Doctest: my_docs.subtract ... ok
test_add_integers (test_mymath.TestAdd) ... ok
test_add_strings (test_mymath.TestAdd) ... ok
test_add_integers (test_mymath_without_main.TestAdd) ... ok
test_add_on_windows (test_mymath_without_main.TestAdd) ... skipped 'Tested only on windows'
test_add_strings (test_mymath_without_main.TestAdd) ... ok
test_func_1 (test_mymath_without_main.TestAdd) ... skipped 'Got skipped sadly'

----------------------------------------------------------------------
Ran 8 tests in 0.003s

OK (skipped=2)


# References
- [Software development skills for data scientists](http://treycausey.com/software_dev_skills.html)